# Week 4 머신러닝 평가 실습
## 교차검증 · 분류 지표 · 종합 실습

| 차시 | 주제 |
|------|------|
| 차시 1 | 교차검증의 원리 (K-Fold, Stratified K-Fold, cross_val_score) |
| 차시 2 | 분류 평가지표와 혼동행렬 (Confusion Matrix, Precision, Recall, F1) |
| 차시 3 | 평가 실습 (confusion_matrix, classification_report, cross_val_score 종합) |

**실습 데이터** : `student_learning_data_kr.csv` (학생 학습 행동 데이터, 14,003행 × 14열)  
**목표 변수** : `최종성적` (A / B / C / D — 4-클래스 분류)


---
## 0. 라이브러리 임포트

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier

---
## 1 단계 │ 데이터 로드 및 전처리

### 컬럼 구성

| 컬럼명 | 설명 | 타입 | 처리 방식 |
|--------|------|------|-----------|
| 주당학습시간 | 주당 학습 시간 | 수치형 | 그대로 사용 |
| 출석률 | 출석률(%) | 수치형 | 그대로 사용 |
| 비교과활동 | 비교과 활동 참여 여부 | 이진 범주형 | 참여→1, 미참여→0 |
| 학습동기 | 학습 동기 수준 | 순서형 | 낮음→0, 보통→1, 높음→2 |
| 인터넷접근 | 인터넷 접근 가능 여부 | 이진 범주형 | 있음→1, 없음→0 |
| 성별 | 성별 | 이진 범주형 | 남→1, 여→0 |
| 나이 | 나이 | 수치형 | 그대로 사용 |
| 선호학습유형 | 선호하는 학습 유형 | 명목형(4종) | 원-핫 인코딩 |
| 온라인강좌수 | 수강 중인 온라인 강좌 수 | 수치형 | 그대로 사용 |
| 토론참여 | 토론 참여 여부 | 이진 범주형 | 참여→1, 미참여→0 |
| 과제완료율 | 과제 완료율(%) | 수치형 | 그대로 사용 |
| 시험점수 | 시험 점수 | 수치형 | 그대로 사용 |
| 스트레스수준 | 스트레스 수준 | 순서형 | 낮음→0, 보통→1, 높음→2 |
| **최종성적** | **목표 변수** | **4-클래스** | **A/B/C/D** |

In [2]:
# ── 데이터 로드 ──────────────────────────────────────────────────
url = "https://raw.githubusercontent.com/SJ-EduLab/ML-for-Education/main/week01/student_learning_data_kr.csv"
df = pd.read_csv(url)

print(f"데이터 크기 : {df.shape[0]}행 × {df.shape[1]}열")
print()
print(df.head(3))

데이터 크기 : 14003행 × 14열

   주당학습시간  출석률 비교과활동 학습동기 인터넷접근 성별  나이 선호학습유형  온라인강좌수 토론참여  과제완료율  시험점수  \
0      19   64   미참여   낮음    있음  여  19  읽기쓰기형       8   참여     59    40   
1      19   64   미참여   낮음    있음  여  23    시각형      16  미참여     90    66   
2      19   64   미참여   낮음    있음  여  28  운동감각형      19  미참여     67    99   

  스트레스수준 최종성적  
0     낮음    D  
1     낮음    C  
2     낮음    A  


In [3]:
# ── 목표 변수 분포 확인 ──────────────────────────────────────────
print("최종성적 분포:")
grade_counts = df["최종성적"].value_counts().sort_index()
for grade, cnt in grade_counts.items():
    ratio = cnt / len(df) * 100
    print(f"  {grade} : {cnt:>5}개  ({ratio:.1f}%)")
print(f"  합계: {len(df)}개")

최종성적 분포:
  A :  3832개  (27.4%)
  B :  3310개  (23.6%)
  C :  3618개  (25.8%)
  D :  3243개  (23.2%)
  합계: 14003개


In [4]:
# ── 전처리 ───────────────────────────────────────────────────────

# 1) 이진 범주형 → 0 / 1
df["비교과활동"] = df["비교과활동"].map({"미참여": 0, "참여": 1})
df["인터넷접근"] = df["인터넷접근"].map({"없음": 0, "있음": 1})
df["성별"]      = df["성별"].map({"여": 0, "남": 1})
df["토론참여"]  = df["토론참여"].map({"미참여": 0, "참여": 1})

# 2) 순서형 범주형 → 정수 (낮음<보통<높음)
ordinal_map = {"낮음": 0, "보통": 1, "높음": 2}
df["학습동기"]    = df["학습동기"].map(ordinal_map)
df["스트레스수준"] = df["스트레스수준"].map(ordinal_map)

# 3) 명목형 → 원-핫 인코딩 (선호학습유형: 시각형/청각형/읽기쓰기형/운동감각형)
df = pd.get_dummies(df, columns=["선호학습유형"], drop_first=False)

print("전처리 후 컬럼:")
print(df.columns.tolist())
print()
print(df.head(3))

전처리 후 컬럼:
['주당학습시간', '출석률', '비교과활동', '학습동기', '인터넷접근', '성별', '나이', '온라인강좌수', '토론참여', '과제완료율', '시험점수', '스트레스수준', '최종성적', '선호학습유형_시각형', '선호학습유형_운동감각형', '선호학습유형_읽기쓰기형', '선호학습유형_청각형']

   주당학습시간  출석률  비교과활동  학습동기  인터넷접근  성별  나이  온라인강좌수  토론참여  과제완료율  시험점수  스트레스수준  \
0      19   64      0     0      1   0  19       8     1     59    40       0   
1      19   64      0     0      1   0  23      16     0     90    66       0   
2      19   64      0     0      1   0  28      19     0     67    99       0   

  최종성적  선호학습유형_시각형  선호학습유형_운동감각형  선호학습유형_읽기쓰기형  선호학습유형_청각형  
0    D       False         False          True       False  
1    C        True         False         False       False  
2    A       False          True         False       False  


In [5]:
# ── 특성(X) / 목표(y) 분리 ──────────────────────────────────────
X = df.drop(columns=["최종성적"])
y = df["최종성적"]

print(f"X shape : {X.shape}")
print(f"y shape : {y.shape}")
print(f"클래스  : {sorted(y.unique())}")

X shape : (14003, 16)
y shape : (14003,)
클래스  : ['A', 'B', 'C', 'D']


---
## 2 단계 │ cross_val_score 기본 사용법
**[차시 1]**

- `X, y` 전체 데이터를 그대로 전달 — `train_test_split` 불필요
- `cv=5` → 5-Fold 교차검증 (내부에서 자동 분할·학습·평가 × 5회)
- 결과를 **"평균 ± 표준편차"** 형식으로 보고

> **다중 클래스 주의** : `최종성적`은 A/B/C/D 4-클래스이므로  
> `scoring='accuracy'` 는 그대로 사용 가능하지만,  
> F1·재현율·정밀도는 `_macro` / `_weighted` 접미사가 필요합니다.

In [6]:
# cross_val_score 기본 사용법 (정확도)
model = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")

print("각 폴드 점수 :", scores.round(4))
print(f"평균         : {scores.mean():.4f}")
print(f"표준편차     : {scores.std():.4f}")
print(f"보고 형식    : {scores.mean():.4f} ± {scores.std():.4f}")

각 폴드 점수 : [0.9068 0.8847 0.8918 0.9039 0.8975]
평균         : 0.8970
표준편차     : 0.0080
보고 형식    : 0.8970 ± 0.0080


---
## 3 단계 │ 실습 ① 데이터 준비 확인
**[차시 3]**

이진 분류(0/1) 대신 **4-클래스 분류(A/B/C/D)** 를 사용합니다.  
클래스 비율이 비교적 균형(약 23~27%)이므로 정확도도 유효한 지표입니다.

In [7]:
# 데이터 준비 최종 확인
print(f"클래스       : {sorted(y.unique())}")
print()
for grade in sorted(y.unique()):
    cnt = (y == grade).sum()
    print(f"  {grade} 클래스 수: {cnt}개")
print(f"  전체       : {len(y)}개")

클래스       : ['A', 'B', 'C', 'D']

  A 클래스 수: 3832개
  B 클래스 수: 3310개
  C 클래스 수: 3618개
  D 클래스 수: 3243개
  전체       : 14003개


---
## 4 단계 │ 실습 ① 혼동행렬 (Confusion Matrix)
**[차시 3]**

```
confusion_matrix(y_test, predictions)  ← 인자 순서 중요!
  행(row) = 실제 클래스
  열(col) = 예측 클래스
```

4-클래스이므로 혼동행렬은 **4 × 4** 행렬이 됩니다.  
각 행에서 대각선 값 = 올바르게 예측한 수 (TP),  
비대각선 값 = 잘못 예측한 수 (FP 또는 FN).

In [8]:
# 분리 + 모델 학습
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y  # stratify: 클래스 비율 유지
)

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# 혼동행렬 출력
CLASS_ORDER = ["A", "B", "C", "D"]   # 행/열 순서 고정
cm = confusion_matrix(y_test, predictions, labels=CLASS_ORDER)

print("혼동행렬 (행=실제, 열=예측):")
print(f"{'':>6}", "  ".join(f"예측{c}" for c in CLASS_ORDER))
for i, row_label in enumerate(CLASS_ORDER):
    row_str = "  ".join(f"{cm[i, j]:>5}" for j in range(len(CLASS_ORDER)))
    print(f"실제{row_label} {row_str}")

혼동행렬 (행=실제, 열=예측):
       예측A  예측B  예측C  예측D
실제A  1120     30      0      0
실제B    32    924     37      0
실제C     0     21   1035     29
실제D     0      0     33    940


In [9]:
# 클래스별 TP / FN / FP 요약
print("클래스별 혼동행렬 요약:")
print(f"{'클래스':>4}  {'TP':>5}  {'FN':>5}  {'FP':>5}  {'재현율':>6}  {'정밀도':>6}")
print("-" * 42)
for i, cls in enumerate(CLASS_ORDER):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp    # 실제 해당 클래스 중 틀린 것
    fp = cm[:, i].sum() - tp    # 예측이 해당 클래스인데 실제는 다른 것
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    print(f"  {cls:>2}  {tp:>5}  {fn:>5}  {fp:>5}  {recall:>6.3f}  {precision:>6.3f}")

클래스별 혼동행렬 요약:
 클래스     TP     FN     FP     재현율     정밀도
------------------------------------------
   A   1120     30     32   0.974   0.972
   B    924     69     51   0.931   0.948
   C   1035     50     70   0.954   0.937
   D    940     33     29   0.966   0.970


---
## 5 단계 │ 실습 ① classification_report 전체 출력
**[차시 3]**

| 항목 | 설명 |
|------|------|
| `target_names` | 클래스 이름을 레이블로 표시 |
| `macro avg` | 클래스 크기 무시, 단순 평균 (소수 클래스에 동일 가중치) |
| `weighted avg` | support 비례 가중 평균 |
| `support` | 각 클래스의 실제 데이터 수 |

In [10]:
# classification_report: 모든 지표를 클래스별로 한 번에 출력
report = classification_report(
    y_test,
    predictions,
    labels=CLASS_ORDER,
    target_names=["A (최우수)", "B (우수)", "C (보통)", "D (미흡)"],
)
print(report)

print("해석 포인트:")
print("  · 각 성적 등급별 재현율 = '해당 등급 학생 중 몇 %를 올바르게 분류했는가'")
print("  · macro avg  : 클래스 비율 무시, 균등 평균")
print("  · weighted avg: support 비례 평균 (클래스 균형이면 macro ≈ weighted)")

              precision    recall  f1-score   support

     A (최우수)       0.97      0.97      0.97      1150
      B (우수)       0.95      0.93      0.94       993
      C (보통)       0.94      0.95      0.95      1085
      D (미흡)       0.97      0.97      0.97       973

    accuracy                           0.96      4201
   macro avg       0.96      0.96      0.96      4201
weighted avg       0.96      0.96      0.96      4201

해석 포인트:
  · 각 성적 등급별 재현율 = '해당 등급 학생 중 몇 %를 올바르게 분류했는가'
  · macro avg  : 클래스 비율 무시, 균등 평균
  · weighted avg: support 비례 평균 (클래스 균형이면 macro ≈ weighted)


---
## 6 단계 │ 실습 ② 5-Fold 교차검증 실행
**[차시 3]**

- `X, y` 전체 데이터를 전달 (`train_test_split` 없이)
- `fit`하지 않은 새 모델 객체를 전달 → 내부에서 5번 반복
- 결과 보고: **"0.XXXX ± 0.XXXX"** 형식

In [11]:
# fit하지 않은 새 모델 객체 전달 — cross_val_score가 5번 fit+predict
model_cv = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(model_cv, X, y, cv=5)  # scoring 기본값='accuracy'

print(f"각 폴드 정확도: {scores.round(4)}")
print(f"평균          : {scores.mean():.4f}")
print(f"표준편차      : {scores.std():.4f}")
print(f"보고 형식     : {scores.mean():.4f} ± {scores.std():.4f}")
print()
print("해석:")
print(f"  · 표준편차 {scores.std():.4f} → 폴드 간 성능 변동 정도")
print("  · 단순 train_test_split 1회보다 안정적인 성능 추정")

각 폴드 정확도: [0.9068 0.8847 0.8918 0.9039 0.8975]
평균          : 0.8970
표준편차      : 0.0080
보고 형식     : 0.8970 ± 0.0080

해석:
  · 표준편차 0.0080 → 폴드 간 성능 변동 정도
  · 단순 train_test_split 1회보다 안정적인 성능 추정


---
## 7 단계 │ 실습 ② scoring 인자 — 다양한 지표로 교차검증
**[차시 3]**

### 다중 클래스에서의 scoring 문자열

| 지표 | 이진 분류 | **다중 클래스** |
|------|-----------|----------------|
| 정확도 | `'accuracy'` | `'accuracy'` (동일) |
| F1 | `'f1'` | `'f1_macro'` 또는 `'f1_weighted'` |
| 재현율 | `'recall'` | `'recall_macro'` 또는 `'recall_weighted'` |
| 정밀도 | `'precision'` | `'precision_macro'` 또는 `'precision_weighted'` |

In [12]:
model_cv = KNeighborsClassifier(n_neighbors=5)

# 정확도 교차검증 (기본값)
scores_acc = cross_val_score(model_cv, X, y, cv=5, scoring="accuracy")
print(f"정확도        : {scores_acc.mean():.4f} ± {scores_acc.std():.4f}")

# F1 교차검증 — 다중 클래스: f1_macro 사용
scores_f1 = cross_val_score(model_cv, X, y, cv=5, scoring="f1_macro")
print(f"F1 (macro)    : {scores_f1.mean():.4f} ± {scores_f1.std():.4f}")

# 재현율 교차검증 — 다중 클래스: recall_macro 사용
scores_rec = cross_val_score(model_cv, X, y, cv=5, scoring="recall_macro")
print(f"재현율 (macro): {scores_rec.mean():.4f} ± {scores_rec.std():.4f}")

print()
print("해석:")
print("  · macro  : 클래스 비율 무시, 각 클래스 점수의 단순 평균")
print("  · weighted: support 비례 평균 (불균형 데이터에서 macro와 차이 발생)")
print("  · 이진 분류의 'f1', 'recall' → 다중 분류에서는 'f1_macro', 'recall_macro' 사용")

정확도        : 0.8970 ± 0.0080
F1 (macro)    : 0.8960 ± 0.0082
재현율 (macro): 0.8957 ± 0.0081

해석:
  · macro  : 클래스 비율 무시, 각 클래스 점수의 단순 평균
  · weighted: support 비례 평균 (불균형 데이터에서 macro와 차이 발생)
  · 이진 분류의 'f1', 'recall' → 다중 분류에서는 'f1_macro', 'recall_macro' 사용


---
## 8 단계 │ 회귀 모델 교차검증 (neg_MSE 주의)
**[차시 3]**

> `scoring='neg_mean_squared_error'` 는 **음수** 로 반환됩니다.  
> scikit-learn 컨벤션: 모든 scoring은 '높을수록 좋음' 방향으로 통일.  
> MSE는 낮을수록 좋으므로 부호를 뒤집어 음수로 반환 → `-scores` 로 복원.

In [13]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

data2   = load_diabetes()
model_r = LinearRegression()

# R²로 교차검증 (기본값)
scores_r2 = cross_val_score(model_r, data2.data, data2.target, cv=5)
print(f"R²  : {scores_r2.mean():.4f} ± {scores_r2.std():.4f}")

# MSE로 교차검증 — 결과가 음수로 반환됨
scores_mse = cross_val_score(
    model_r,
    data2.data,
    data2.target,
    cv=5,
    scoring="neg_mean_squared_error",
)
mse_values = -scores_mse   # 부호 뒤집기 → 실제 MSE 값
print(f"MSE : {mse_values.mean():.2f} ± {mse_values.std():.2f}")

print()
print("주의: neg_mean_squared_error 결과는 음수 → -scores 로 변환 필요")

R²  : 0.4823 ± 0.0493
MSE : 2993.08 ± 150.77

주의: neg_mean_squared_error 결과는 음수 → -scores 로 변환 필요


---
## 9 단계 │ 판단 토론 — 이 모델을 쓸 것인가?
**[차시 3]**

### 시나리오: 학습부진(D등급) 학생 조기 선별 시스템

- 전체 학생 중 **D등급(미흡)** 학생을 조기에 선별하여 지원
- 이 경우 **D등급 재현율** = 핵심 지표
- FN(D등급 학생을 놓침) > FP(다른 등급을 D로 오분류) 관점

In [14]:
# D등급 재현율 집중 분석
report_dict = classification_report(
    y_test,
    predictions,
    labels=CLASS_ORDER,
    output_dict=True,
)

d_precision = report_dict["D"]["precision"]
d_recall    = report_dict["D"]["recall"]
d_f1        = report_dict["D"]["f1-score"]
d_support   = int(report_dict["D"]["support"])

print("D등급(미흡) 학생 선별 성능:")
print(f"  정밀도 (Precision) : {d_precision:.3f}")
print(f"  재현율 (Recall)    : {d_recall:.3f}")
print(f"  F1 점수            : {d_f1:.3f}")
print(f"  실제 D등급 수      : {d_support}명")
print()

found   = int(d_support * d_recall)
missed  = d_support - found
print(f"  → 실제 D등급 {d_support}명 중 {found}명 탐지, {missed}명 놓침")
print()
print("판단 근거:")
print("  ① 시스템 목적 = D등급 학생 조기 발견 → FN이 가장 비싼 실수")
print("  ② FP(다른 등급을 D로 오분류)는 교사 2차 확인으로 복구 가능")
print("  ③ FN(D등급을 놓침)은 학생이 지원 없이 방치됨")
print("  → 재현율을 최우선 지표로 사용")

D등급(미흡) 학생 선별 성능:
  정밀도 (Precision) : 0.970
  재현율 (Recall)    : 0.966
  F1 점수            : 0.968
  실제 D등급 수      : 973명

  → 실제 D등급 973명 중 940명 탐지, 33명 놓침

판단 근거:
  ① 시스템 목적 = D등급 학생 조기 발견 → FN이 가장 비싼 실수
  ② FP(다른 등급을 D로 오분류)는 교사 2차 확인으로 복구 가능
  ③ FN(D등급을 놓침)은 학생이 지원 없이 방치됨
  → 재현율을 최우선 지표로 사용


---
## 10  단계 │ 4주차 종합 정리

| 차시 | 핵심 개념 | 코드 |
|------|-----------|------|
| 차시 1 | 단순 split 한계 → K-Fold 교차검증 | `cross_val_score(model, X, y, cv=5)` |
| 차시 2 | 정확도 함정 → 혼동행렬 · Precision · Recall · F1 | `confusion_matrix`, `classification_report` |
| 차시 3 | 코드 실행 + 지표 선택 + 판단 토론 | 전체 통합 |

### 다중 클래스(4-class) 핵심 변경사항

```python
# 이진 분류 → 다중 클래스 변경 시 주의
scoring='f1'       → scoring='f1_macro'        # F1
scoring='recall'   → scoring='recall_macro'    # 재현율
scoring='precision'→ scoring='precision_macro' # 정밀도

# confusion_matrix: labels 인자로 클래스 순서 고정 권장
confusion_matrix(y_test, predictions, labels=['A','B','C','D'])
```

### 4주차 핵심 메시지

> **올바른 평가 = 올바른 방법(교차검증) + 올바른 지표(맥락에 따라) + 올바른 판단(전문성)**

In [15]:
# 최종 종합 요약 출력
model_final = KNeighborsClassifier(n_neighbors=5)

# 교차검증 결과
cv_acc = cross_val_score(model_final, X, y, cv=5, scoring="accuracy")
cv_f1  = cross_val_score(model_final, X, y, cv=5, scoring="f1_macro")
cv_rec = cross_val_score(model_final, X, y, cv=5, scoring="recall_macro")

print("=" * 55)
print("4주차 최종 종합 결과 (student_learning_data_kr.csv)")
print("=" * 55)
print(f"데이터       : {len(X)}행 × {X.shape[1]}열")
print(f"모델         : k-NN (k=5)")
print(f"평가 방법    : Stratified 5-Fold 교차검증")
print()
print("[ 교차검증 결과 ]")
print(f"  정확도        : {cv_acc.mean():.4f} ± {cv_acc.std():.4f}")
print(f"  F1 (macro)    : {cv_f1.mean():.4f} ± {cv_f1.std():.4f}")
print(f"  재현율 (macro): {cv_rec.mean():.4f} ± {cv_rec.std():.4f}")
print()
print("[ 단순 split 결과 (참고) ]")
acc_single = (predictions == y_test).mean()
print(f"  정확도 (1회) : {acc_single:.4f}")
print()
print("→ 교차검증 결과가 단일 split보다 더 신뢰할 수 있는 성능 추정")

4주차 최종 종합 결과 (student_learning_data_kr.csv)
데이터       : 14003행 × 16열
모델         : k-NN (k=5)
평가 방법    : Stratified 5-Fold 교차검증

[ 교차검증 결과 ]
  정확도        : 0.8970 ± 0.0080
  F1 (macro)    : 0.8960 ± 0.0082
  재현율 (macro): 0.8957 ± 0.0081

[ 단순 split 결과 (참고) ]
  정확도 (1회) : 0.9567

→ 교차검증 결과가 단일 split보다 더 신뢰할 수 있는 성능 추정
